In [258]:
from yummly import Client
import json
import requests
import pandas as pd
import numpy as np 
import re

In [259]:
# API call for the first 500 BB recipes labeled as such only!
header= {'X-Yummly-App-ID':'79663a75', 'X-Yummly-App-Key':'02b233108f476f3110e0f65437c4d6dd'}
url='http://api.yummly.com/v1/api/recipes?'
parameters={
            'allowedCourse[]':'course^course-Breakfast and Brunch',
            'excludedCourse[]': ['course^course-Main Dishes','course^course-Appetizers', 'course^course-Salads', 'course^course-Lunch',
                                'course^course-Side Dishes','course^course-Desserts','course^course-Breads',
                                 'course^course-Soups', 'course^course-Beverages', 'course^course-Condiments and Sauces',
                                'course^course-Cocktails', 'course^course-Snacks'],
            'maxResult': 500,
            'start': 0
            }

response=requests.get(url, headers = header, params = parameters)

In [260]:
response.status_code

200

In [261]:
BB=response.json()
print type(BB)
print BB.keys()

<type 'dict'>
[u'matches', u'totalMatchCount', u'attribution', u'facetCounts', u'criteria']


In [262]:
#only interrested in the information under matches. 
print len(BB['matches'])
print type(BB['matches'])
print BB['matches'][0].keys()

500
<type 'list'>
[u'flavors', u'rating', u'totalTimeInSeconds', u'ingredients', u'smallImageUrls', u'sourceDisplayName', u'recipeName', u'attributes', u'id', u'imageUrlsBySize']


In [263]:
#checkout one recipe
BB_matches=BB['matches']
BB_matches[499]

{u'attributes': {u'course': [u'Breakfast and Brunch']},
 u'flavors': {u'bitter': 0.6666666666666666,
  u'meaty': 0.3333333333333333,
  u'piquant': 0.0,
  u'salty': 0.8333333333333334,
  u'sour': 0.16666666666666666,
  u'sweet': 0.16666666666666666},
 u'id': u'Baked-French-Toast-Casserole-1636754',
 u'imageUrlsBySize': {u'90': u'https://lh3.googleusercontent.com/pSQ8-zmDr_c5asdCRjpX_9wCp5TWzzsKu25wNyJHeJNezAGS8_0xqxOmVaD1k2iH8RPYrbjFxqRQ3kPh6N7O=s90-c'},
 u'ingredients': [u'french bread',
  u'eggs',
  u'milk',
  u'vanilla',
  u'salt',
  u'unsalted butter'],
 u'rating': 4,
 u'recipeName': u'Baked French Toast Casserole',
 u'smallImageUrls': [u'https://lh3.googleusercontent.com/mUYnOpHQhuIZFNCXgvgbPp0LU9phviIqIHa3FEHgVR8HIea86HsQiXxlKln5arT_ppDpPOXIzNDdo1n-qS11=s90'],
 u'sourceDisplayName': u'Life Currents',
 u'totalTimeInSeconds': 3600}

In [264]:
#forming lists to create dataframes of the features we want. 
main_list = []
ingredients_list = []
attributes_list = []
flavors_list = []

for food in BB_matches:

    _d1 = {}
    _d1['id'] = food['id']
    _d1['rating'] = food['rating']
    _d1['recipeName'] = food['recipeName']
    _d1['sourceDisplayName'] = food['sourceDisplayName']
    
    main_list.append(_d1)
    
    _d2 = {}
    _d2['id'] =food['id']
    for i in food['ingredients']:
        i = i.lower() # additional code to conver to lowercase
        i = re.sub(r'\d+%\s', '', i) # additional code to remove 1%, 2%, etc
        i = re.sub(r'\xae', '', i) # remove '\xae' characters
        _d2[i] = 1
    ingredients_list.append(_d2)

    _d3 = {}
    _d3['id'] = food['id']
    for k, v in food['attributes'].items():
        for i in v:
            _d3[i] = 1
    attributes_list.append(_d3)
 

In [265]:
flavors_dict = {}

for food in BB_matches:
    flavors_dict[food.get('id')] = food.get('flavors')

In [267]:
# read in dictionary for course and cuisine and create list of possible values for each
course_df = pd.read_csv('course_headers.csv', names=['course'])
cuisine_df = pd.read_csv('cuisine_headers.csv', names=['cuisine'])

course_list = course_df.course
cuisine_list= cuisine_df.cuisine


IOError: File course_headers.csv does not exist

In [206]:
#create dictionary of cuisine and course for each recipe
course_dict={}
cuisine_dict={}
for food in BB_matches:
    course_dict[food.get('id')]= food['attributes'].get('course')
    cuisine_dict[food.get('id')]= food['attributes'].get('cuisine')
    
_courses= {}

for k, v in course_dict.iteritems():
    course_val = {}
    for course in course_list:
        try:
            if course in v :
                course_val[course] = 1
            else:
                course_val[course] = 0
        except TypeError:
            course_val[course] = 0
    
        _courses[k] = course_val
        
_cuisines= {}       

for k, v in cuisine_dict.iteritems():
    cuisine_val = {}
    for course in cuisine_list:
        try:
            if course in v :
                cuisine_val[course] = 1
            else:
                cuisine_val[course] = 0
        except TypeError:
            cuisine_val[course] = 0
    
        _cuisines[k] = cuisine_val


In [207]:
#get list of recipe ids
recipe_ids=[]
for recipe in BB_matches:
    recipe_ids.append(recipe['id'])

In [208]:
# second api call to get other features for each recipe
key_id= '_app_id=79663a75&_app_key=02b233108f476f3110e0f65437c4d6dd'
url='http://api.yummly.com/v1/api/recipe/'

In [209]:
# retrieve other features for all recipes

def get_recipe(_id):
    response = requests.get(url + _id + '?' + key_id)
    return response.json()

recipes=[]
for _id in recipe_ids :
    recipes.append(get_recipe(_id))

In [210]:
response.status_code

200

In [211]:
print len(recipes)
print recipes[1].keys()

500
[u'totalTime', u'ingredientLines', u'attribution', u'name', u'rating', u'numberOfServings', u'yield', u'nutritionEstimates', u'source', u'flavors', u'images', u'attributes', u'id', u'totalTimeInSeconds']


In [212]:
#for each recipe create a new dictionary of selected attributes and append into a list

recipe_details=[]
for recipe in recipes:
    _dict={}
    #import pdb; pdb.set_trace()
    _dict['id']=recipe['id']
    _dict['ingredientCount']= len(recipe['ingredientLines'])
    _dict['numberOfServings']= recipe['numberOfServings']
    if 'prepTimeInSeconds' in recipe.keys():
        _dict['prepTimeInSeconds']= recipe['prepTimeInSeconds']
    else:
        _dict['prepTimeInSeconds']= None
    if 'cookTimeInSeconds' in recipe.keys():
        _dict['cookTimeInSeconds']= recipe['cookTimeInSeconds']
    else:
        _dict['cookTimeInSeconds']= None
    _dict['totalTimeInSeconds']=recipe['totalTimeInSeconds']
    
    
    recipe_details.append(_dict)
        

In [256]:
#create dataframes, arrange column index and save into csv
df_main = pd.DataFrame(main_list)
df_main.to_csv('BB_main.csv', encoding ='utf-8')

df_ingredients = pd.DataFrame(ingredients_list)
df_ingredients = df_ingredients.fillna(0)
cols = list(df_ingredients)
cols.insert(0, cols.pop(cols.index('id')))
df_ingredients= df_ingredients.ix[:,cols]
df_ingredients.to_csv('BB_ingredients.csv', encoding ='utf-8')

df_attributes = pd.DataFrame(attributes_list)
df_attributes = df_attributes.fillna(0)
cols = list(df_attributes)
cols.insert(0, cols.pop(cols.index('id')))
df_attributes = df_attributes.ix[:,cols]
df_attributes.to_csv('BB_attributes.csv')

df_flavors = pd.DataFrame(flavors_dict).transpose()
df_flavors.reset_index(level=0, inplace=True)
df_flavors.to_csv('BB_flavors.csv')

df_courses = pd.DataFrame(_courses).transpose()
df_courses.reset_index(level=0, inplace=True)
df_courses.to_csv('BB_courses.csv')

df_cuisines = pd.DataFrame(_cuisines).transpose()
df_cuisines.reset_index(level=0, inplace=True)
df_cuisines.to_csv('BB_cuisines.csv')

df_details=pd.DataFrame(recipe_details)
cols = list(df_details)
cols.insert(0, cols.pop(cols.index('id')))
df_details=df_details.ix[:,cols]
df_details.to_csv('BB_details.csv')

In [235]:
ingredients = df_ingredients.sum(axis=0).sort_values(ascending=False)


In [243]:
top_ingredients = ingredients[:40]

In [244]:
type(top_ingredients)

pandas.core.series.Series

In [245]:
top_ingredients.to_csv('top_ing.csv')

In [257]:
df_ingredients.columns

Index([                                                            u'id',
                                                  u'2% reduced-fat milk',
                                                       u'Chobani Yogurt',
                                                        u'Crunch Cereal',
                                                      u'English muffins',
                                                       u'Himalayan salt',
                  u'Honeysuckle White® Turkey Breakfast Sausage Patties',
                                                            u'Jarlsberg',
       u'KRAFT Shredded Pepper Jack Cheese with a TOUCH OF PHILADELPHIA',
                                  u'KRAFT Shredded Sharp Cheddar Cheese',
       ...
                                                    u'whole wheat flour',
                                             u'whole wheat pastry flour',
                                                u'whole wheat tortillas',
                           